In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd


In [3]:
from diem.io import read_diem_bed
from diem.io import write_polarized_bed
from diem import diemtype as dt

In [4]:
bedIn = '/home/derek/Dropbox/methods/diempy_example_data/CM001010.3_variants.vcf.diem_input.bed'
metaIn = '/home/derek/Dropbox/methods/diempy_example_data/CM001010.3_variants.vcf.diem_meta.bed'
bedOut = '/home/derek/Dropbox/methods/diempy_example_data/CM001010.3_variants.vcf.diem_output.bed'
d = read_diem_bed(bedIn,metaIn)

When polarizing a diemtypeobject, it RETURNS a POLARIZED COPY

In [5]:
dPol = d.polarize(ncores = 3,boolTestData=True)

convert state matrix to Marray
initializing test polarity
running EM algorithm in parallel using diem_parallel with  3  cores
starting
using  3  CPUs
iteration  0   num 0 polarity =  480698   num 1 polarity =  480697
time to flip:  1.30495285987854
iteration  1   num 0 polarity =  254073   num 1 polarity =  707322
time to flip:  1.1472415924072266
iteration  2   num 0 polarity =  246685   num 1 polarity =  714710
time to flip:  1.0958716869354248
iteration  3   num 0 polarity =  240551   num 1 polarity =  720844
time to flip:  1.0415661334991455
iteration  4   num 0 polarity =  237606   num 1 polarity =  723789
time to flip:  1.0077171325683594
iteration  5   num 0 polarity =  236900   num 1 polarity =  724495
time to flip:  1.0618605613708496
iteration  6   num 0 polarity =  236889   num 1 polarity =  724506
time to flip:  1.058382272720337
iteration  7   num 0 polarity =  236904   num 1 polarity =  724491
time to flip:  0.8183643817901611
iteration  8   num 0 polarity =  236900   num

This is a (currently slow way) to output that polarization information to an output.bed file.  It reads in the original, appends the additional columns, then writes it all out again.

This makes it easy because even if we have somehow resorted the individuals by hybrid index, that has no effect on the output file. It also means we keep all the state information that is removed (that is, converted to '_' or 0 state internal to diemtype object)

In [6]:
write_polarized_bed(bedIn,bedOut,dPol)

Thresholding RETURNS A COPY of the diemtype object with sites removed that are below threshold

In [ ]:
dThresh = dPol.apply_threshold(-20)


proportion of each chromosome retained after thresholding:
[np.float64(0.8675965654075588)]
thresholding done
hybrid indices have likely changed, but have not been updated
to update HIs without re-sorting, update the attribute a.HIs = a.computeHIs()
You may also call the sort() method on the resulting data if you wish to both re-compute and resort


SORTING IS DONE IN-PLACE
meaning that the diemtypeobject itself is sorted. Nothing is returned.
To obtain a sorted *copy* you must first copy it then sort as follows:

In [11]:
dSorted = dThresh.copy()
dSorted.sort()

new HIs computer and individuals resorted by HI


Smoothing RETURNS A COPY

In [13]:
dSmoothed = dSorted.smooth(1e-4,parallel=True)

smoothing the data with scale  0.0001
using parallel smoothing
smoothing done, but not resorting by HI. You may call the sort() method on the resulting data if you wish to resort


Constructing contigs adds this information to the diemtype objects
Meaning that it DOES NOT RETURN A COPY


In [ ]:
dSmoothed.create_contig_matrix()

contig matrix created and stored in self.contigMatrix


We provide a path to a *folder* where we want the intervals information to be written to individual-specific bed files. 

In [17]:
outIntervalsFolder = '/home/derek/Dropbox/methods/diempy_example_data/CM001010_intervals/'
dSmoothed.intervals_to_bed(outIntervalsFolder)

BED files created in directory:  /home/derek/Dropbox/methods/diempy_example_data/CM001010_intervals/
